### PHASE 3 — DATA PREPARATION (CLEANING & TRANSFORMATION)

In [8]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

In [9]:
online_retail = pd.read_excel('online_retail.xlsx')

In [10]:
# Filtering out all transactions with negative or zero quantity.
filtered_quantity = online_retail[online_retail['Quantity']>0]

In [12]:
# Removing all rows with UnitPrice < 0.
filtered_unitprice = filtered_quantity[filtered_quantity['UnitPrice'] >= 0]

In [ ]:
# Ensure 'InvoiceNo' column is of string type
# Use .loc to modify the column (if you want to modify in place)
filtered_unitprice.loc[:, 'InvoiceNo'] = filtered_unitprice['InvoiceNo'].astype(str)

# Remove invoice cancellations (InvoiceNo beginning with "C").
filtered_invoice_cancellations = filtered_unitprice[~filtered_unitprice['InvoiceNo'].str.startswith('C')]

In [17]:
# Drop rows where CustomerID is missing.
filtered_custID = filtered_invoice_cancellations.dropna(subset=['CustomerID'])

In [19]:
# Remove duplicate records
dropped_duplicates = filtered_custID.drop_duplicates()

In [25]:
# Create new fields:
# Revenue = Quantity × UnitPrice
dropped_duplicates.loc[:, 'Revenue'] = dropped_duplicates['Quantity'] * dropped_duplicates['UnitPrice']

In [ ]:
# Month extracted from InvoiceDate
dropped_duplicates.loc[:, 'Month'] = dropped_duplicates['InvoiceDate'].dt.month

In [ ]:
# Year extracted from InvoiceDate
dropped_duplicates.loc[:, 'Year'] = dropped_duplicates['InvoiceDate'].dt.year

In [ ]:
# Day extracted from InvoiceDate
dropped_duplicates.loc[:, 'Day'] = dropped_duplicates['InvoiceDate'].dt.day

In [33]:
dropped_duplicates.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue,Month,Year,Day
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,12,2010,1
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,12,2010,1
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,12,2010,1
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,12,2010,1
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,12,2010,1


#### Store a "cleaned" version of the data in the db

In [35]:
load_dotenv()
db_url = os.getenv("database_url")

In [37]:
engine = create_engine(db_url)
dropped_duplicates.to_sql('clean_online_retail_data', engine, if_exists='replace', index=False, schema='online_retail')

732